## CPSC 477 Final Project Part 1: Augmenting ECTSum With Gemini


We used Gemini 1.0 Pro through Google's Generative AI API to augment the summaries in ECTSum. We noticed that the provided bullet-point style summaries in ECTSum were too concise and failed to capture the plethora of useful information in the earnings call transcripts.

For example, this was the provided summary in ECTSum for `DG_q3_2021.txt`.

```
compname reports q3 earnings per share of $2.08.
q3 earnings per share $2.08.
sees fy sales up about 1 to 1.5 percent.
q3 sales rose 3.9 percent to $8.5 billion.
```

To solve this problem, we prompted Gemini to summarize `DG_q3_2021.txt`. Furthermore, we emphasized that it should incorporate all the information in the provided summary (which we referred to as the *Key Points* in the prompt). 

Here is the prompt that we used:

```
You are a financial advisor tasked with creating a short summary of an earnings call transcript. You only want to summarize or re-iterate points that would be relevant, critical, or informational to someone who wants to skim over the important details of a long transcript.

Below is an earnings call transcript. Please summarize this transcript in one paragraphs using complete sentences. Keep the summary below 500 words. Include relevant information from the Earnings Call Transcript in your summary. Furthermore, it is very important that you incorporate all the information from the Key Points and spread it out throughout your summary.

Earnings Call Transcript:
[ect]

Key Points:
[summary]
```

Note that our prompt contains the phrase `"one paragraphs"`, which was a grammatical mistake we did not catch until after generating all of the summaries. However, upon fixing this grammatical mistake in the prompt and running Gemini, we paradoxically observed that the model largely produced summaries with more than one paragraph. In comparison, our prompt produced a majority of single-paragraph summaries and a lesser amount of summaries that were composed of multiple paragraphs. This is essentially why we kept this prompt. In any case, we concatenated these paragraphs to convert everything into a single-paragraph summary before we fed it into the model.

Here is the Gemini-augmented summary for the `DG_q3_2021.txt` example shown above:

```
Dollar General reported a 3.9% increase in net sales to $8.5 billion for the third quarter, exceeding market expectations. Despite a 0.6% decline in same-store sales, the company remains confident in its competitive position due to a well-performing price index and the significance of $1 price point products to its customers. The retailer has ambitious plans for 2022, including nearly 3,000 real estate projects and the acceleration of its pOpshelf concept, targeting 1,000 stores by fiscal year-end 2025. Additionally, Dollar General announced plans to enter Mexico with up to 10 stores in 2022, recognizing Mexico's compelling expansion opportunities. The company's financial outlook for 2021 includes net sales growth of approximately 1% to 1.5%, and earnings per share in the range of $9.90 to $10.20, representing a 22% to 24% increase over 2019 adjusted earnings per share. Dollar General's key initiatives, including NCI, pOpshelf, and DG Fresh, continue to drive growth, with NCI contributing a 2.5% total comp sales increase in stores and pOpshelf exceeding expectations both in sales and gross margin. The retailer's priority remains capturing growth opportunities, with plans to open 2,980 real estate projects in 2022, including 1,110 new stores, 1,750 remodels, and 120 store relocations.
```

Notice that the prompt's insistence on incorporating the Key Points worked: the response captures that `q3 sales rose 3.9 percent to $8.5 billion` in the first line and `fy sales up about 1 to 1.5 percent` later on. While the generated summary does not include the information of `q3 earnings per share $2.08`, it mentions forecasted `earnings per share in the range of $9.90 to $10.20`. The model would have obtained this piece of information from the following section of the earnings call transcript.

<pre>
For 2021, we now expect the following: Net sales growth of approximately 1% to 1.5%; a same-store sales decline of approximately 3% to 2.5%, but which reflects growth of approximately 13% to 14% on a two-year stack basis; and <b>earnings per share in the range of $9.90 to $10.20</b>, which reflects a compound annual growth rate in the range of 22% to 24% or approximately 21% to 23%, compared to 2019 adjusted earnings per share over a two-year period.
</pre>

We suspect that our strategy of strongly suggesting the model to incorporate the information in the provided summary directly influences it to be more factual and incorporate more statistics. Having more statistics increases the usefulness of the transcript summaries for people seeking to gain insights from them and decreases the amount of superfluous text present.

The final thing we would like to note is that we ended up deleting small amount (~5%) of Gemini's summaries generated when running our scripts below. This is because they had wildly different formats with markdown formatting, bullet-points, or titles (single exception: if the only problem was one title, we simply deleted the title and kept the summary). We provide 7 examples of either `deleted` or `kept_but_deleted_title` summaries in the `gemini_bad_output_examples` directory. These examples serve to elaborate our deletion process to aid those reproducing this part of our project.

In total, we generated summaries with Gemini for 200 randomly chosen transcripts in the `train` fold, 50 in `val`, and 50 in `test`. We added these summaries to those in ECTSum to created an modified version of Mukherjee et al.'s ECTSum benchmark.

In [52]:
# !pip install google-generativeai
import google.generativeai as genai
import random
from dotenv import load_dotenv
import os
import time
env = load_dotenv(override = True)

In [2]:
genai.configure(api_key = os.environ["API_KEY"])
model = genai.GenerativeModel("gemini-pro")

In [101]:
fold = "train"
ect_paths = os.listdir(f"data/final/{fold}/ects")
ect_paths = random.choices(ect_paths, k = 10)

ects = []
summaries = []

for ect_path in ect_paths:
    with open(f"data/final/{fold}/ects/{ect_path}") as f:
        ects.append(f.read().strip())
    with open(f"data/final/{fold}/gt_summaries/{ect_path}") as f:
        summaries.append(f.read().strip())

prompt = "You are a financial advisor tasked with creating a short summary of an earnings call transcript. You only want to summarize or re-iterate points that would be relevant, critical, or informational to someone who wants to skim over the important details of a long transcript.\n\nBelow is an earnings call transcript. Please summarize this transcript in only one paragraph using complete sentences. Keep the summary below 500 words. Include relevant information from the Earnings Call Transcript in your summary. Furthermore, it is very important that you incorporate all the information from the Key Points and spread it out throughout your summary."
for i, ect_path in enumerate(ect_paths):
    if os.path.isfile(f"data/final/{fold}/gemini_summaries/{ect_path}"):
        continue
    response = model.generate_content(f"{prompt}\n\nEarnings Call Transcript:\n{ects[i]}\n\nKey Points:\n{summaries[i]}\n")
    gemini_summary = response.text
    with open(f"data/final/{fold}/gemini_summaries/{ect_path}", "w") as f:
        f.write(gemini_summary)
    

In [103]:
print(len(os.listdir("data/final/train/gemini_summaries")))
print(len(os.listdir("data/final/test/gemini_summaries")))
print(len(os.listdir("data/final/val/gemini_summaries")))

200
50
50
